In [ ]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import math, os

config = tf.ConfigProto()
config.gpu_options.allow_growth = True

In [ ]:
batch_size = 512
z_dim = 10
# NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN = 2135
# NUM_EXAMPLES_PER_EPOCH_FOR_TEST = 536
TOTAL_EXAMPLES = 6434
NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN = int(TOTAL_EXAMPLES * .8)
NUM_EXAMPLES_PER_EPOCH_FOR_TEST = TOTAL_EXAMPLES - int(TOTAL_EXAMPLES * .8)
# Lambda = 0.005 # kl
Lambda1 = 1 # mmd
Lambda2 = .002 # HSIC, fist axis
Lambda3 = .05 # HSIC, rest of axis
prob_to_hsic = 1
Sigma = 1
steps = 18001
K1 =  'IMQ' # for mmd, try IMQ, for IMQ, fix bandwidth
# K1 = 'Gaussian'
K2 =  'Gaussian' # for hsic, try gausian
bandwidth1 = np.sqrt(z_dim) * Sigma # or -1 # IMQ for larger mmd
# bandwidth1 = 2. * z_dim * (Sigma ** 2) 
bandwidth2 = 0


# bandwidth = 0


In [ ]:
tf.reset_default_graph()

# Define some handy network layers
def lrelu(x, rate=0.1):
    return tf.maximum(tf.minimum(x * rate, 0), x)

def conv2d_lrelu(inputs, num_outputs, kernel_size, stride):
    conv = tf.contrib.layers.convolution2d(inputs, num_outputs, kernel_size, stride, 
                                           weights_initializer=tf.contrib.layers.xavier_initializer(),
                                           activation_fn=tf.identity)
    conv = tf.contrib.layers.batch_norm(
          conv, center=True, scale=True,
          epsilon=1e-05, decay=0.9, updates_collections=None, fused=False)
    
    conv = lrelu(conv)
    
    return conv

def conv2d_t_relu(inputs, num_outputs, kernel_size, stride):
    conv = tf.contrib.layers.convolution2d_transpose(inputs, num_outputs, kernel_size, stride,
                                                     weights_initializer=tf.contrib.layers.xavier_initializer(),
                                                     activation_fn=tf.identity)
    
    conv = tf.contrib.layers.batch_norm(
          conv, center=True, scale=True,
          epsilon=1e-05, decay=0.9, updates_collections=None, fused=False)
    
    conv = tf.nn.relu(conv)
    return conv

def fc_lrelu(inputs, num_outputs):
    fc = tf.contrib.layers.fully_connected(inputs, num_outputs,
                                           weights_initializer=tf.contrib.layers.xavier_initializer(),
                                           activation_fn=tf.identity)
    fc = lrelu(fc)
    return fc

def fc_relu(inputs, num_outputs):
    fc = tf.contrib.layers.fully_connected(inputs, num_outputs,
                                           weights_initializer=tf.contrib.layers.xavier_initializer(),
                                           activation_fn=tf.identity)
    fc = tf.nn.relu(fc)
    return fc

def max_pool2d(inputs, kernel_size):
    return tf.contrib.layers.max_pool2d(inputs, kernel_size)
  
def dropout(inputs):
    return tf.contrib.layers.dropout(inputs)
    

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# DATASET = "..."

lidc = np.load(DATASET)

lidc_train_x = lidc['train_x']/255.
lidc_train_y = lidc['train_y']
lidc_test_x = lidc['test_x']/255.
lidc_test_y = lidc['test_y']

print(lidc_train_x.shape)
print(lidc_test_x.shape)

unique, counts = np.unique(lidc_train_y, return_counts=True)
print(dict(zip(unique, counts)))

from matplotlib import pyplot as plt
plt.imshow(lidc_train_x[150], cmap='Greys_r')
plt.grid()
plt.show()

In [ ]:
def lidc_encoder(x, z_dim, reuse=False):
  with tf.device('/gpu:0'):
    with tf.variable_scope('encoder') as en:
        if reuse:
            en.reuse_variables()
        
        conv1 = conv2d_lrelu(x, 64, 4, 2)
        pool1 = max_pool2d(conv1, [2, 2])
#         drop1 = dropout(pool1)
        
        conv2 = conv2d_lrelu(pool1, 128, 4, 2)
        pool2 = max_pool2d(conv2, [2, 2])
#         drop2 = dropout(pool2)

        conv3 = conv2d_lrelu(pool2, 256, 4, 1)
    
        print(conv3)
        flat_z = tf.reshape(conv3, [-1, np.prod(conv3.get_shape().as_list()[1:])])
        
        fc1 = tf.contrib.layers.fully_connected(flat_z, 256, activation_fn=lrelu)
        
        return tf.contrib.layers.fully_connected(fc1, z_dim * 2, activation_fn=tf.identity)
  
def lidc_decoder(z, reuse=False):
  with tf.device('/gpu:0'):
    with tf.variable_scope('decoder') as vs:
        if reuse:
            vs.reuse_variables()
        
        fc2 = tf.contrib.layers.fully_connected(z, 256, activation_fn=lrelu)

        fc3 = fc_relu(fc2, 3*3*256)
        fc3 = tf.reshape(fc3, tf.stack([tf.shape(fc3)[0], 3, 3, 256]))
      
        deconv1 = conv2d_t_relu(fc3, 128, 4, 2)
        deconv2 = conv2d_t_relu(deconv1, 64, 4, 2)
        deconv3 = conv2d_t_relu(deconv2, 32, 4, 2)
        deconv4 = tf.contrib.layers.convolution2d_transpose(deconv3, 1, 4, 2,
                                                     weights_initializer=tf.contrib.layers.xavier_initializer(),
                                                     activation_fn=tf.identity)

        return tf.nn.sigmoid(deconv4)

In [ ]:
def comp_med(X,kernel='Gaussian'):
    H = compute_diff(X,X)
    H = tf.where(tf.greater(H,0))
    if tf.shape(H)[0] == 0:
        return 1
    else:
        h = tf.contrib.distributions.percentile(H, 50.0)
        if kernel == 'Gaussian':
            return tf.sqrt(tf.cast(h/2,tf.float32)) 
            #return np.sqrt(0.5 * h) / np.log(X.shape[0]+1)
        else:
            return tf.sqrt(tf.cast(h,tf.float32))

def compute_diff(X,Y):
    XY = X[:,tf.newaxis,:] - Y
    out = tf.einsum('ijk,ijk->ij',XY,XY)
    return out
    
def compute_gram(X,Y, bandwidth, kernel='Gaussian'):
    H = compute_diff(X,Y)
    if kernel == 'Gaussian':
        return tf.exp(- H / 2 / tf.cast(bandwidth**2, tf.float32)) # (x_size, y_size)
    else: # inverse multiquadratics kernel
        # return c / (c + H) 
        return tf.pow(tf.cast(bandwidth**2,tf.float32) + H, -0.5)

# MMD

def MMD(X, Y, kernel='Gaussian', bandwidth=0):
    
    if bandwidth <= 0: # median heuristic
        bandwidth = comp_med(tf.concat([X,Y],0))

    XX = compute_gram(X,X,bandwidth,kernel)
    YY = compute_gram(Y,Y,bandwidth,kernel)
    XY = compute_gram(X,Y,bandwidth,kernel)
    XX = tf.reduce_mean(XX)
    YY = tf.reduce_mean(YY)
    XY = tf.reduce_mean(XY)
    
    return XX + YY - 2*XY # biased V-stats

In [ ]:
def reparameterize(mean, logvar, random=True):
    if random:
        eps = tf.random_normal(shape=tf.shape(mean))
        return eps * tf.exp(logvar * .5) + mean
    else: # deterministic
        return mean
    
def push_forward(encoder, train_x):
    mean, logvar = tf.split(encoder(train_x,z_dim), num_or_size_splits=2, axis=1)
    return reparameterize(mean,logvar)
    
def compute_kl_loss(mean, logvar):
        
    kl_divergence = 1 + logvar - tf.square(mean) - tf.exp(logvar)
    kl_divergence = -0.5 * tf.reduce_sum(kl_divergence, 1)

    return kl_divergence

In [ ]:
def HSIC(X, Y, kernel='Gaussian', bandwidthX=0, bandwidthY=0, normalized=True): # set normalized to False
    if tf.keras.backend.ndim(X) == 1:
        X = tf.expand_dims(X,axis=1)
    if tf.keras.backend.ndim(Y) == 1:
        Y = tf.expand_dims(Y,axis=1)
    n = tf.shape(X)[0]
    
    if bandwidthX == 0:
        bandwidthX = comp_med(X,kernel)
    if bandwidthY == 0:
        bandwidthY = comp_med(Y,kernel)
        
    K = compute_gram(X,X,bandwidthX,kernel)
    L = compute_gram(Y,Y,bandwidthY,kernel)
    H = tf.eye(n) - (tf.ones([n,n]) / tf.cast(n,tf.float32))
    Kc = tf.matmul(tf.matmul(H,K),H);
    trace = tf.reduce_sum(L*(tf.transpose(Kc)))
    if normalized:
        HKH = tf.norm(tf.matmul(tf.matmul(H,K),H),ord='fro',axis=[0,1])
        HLH = tf.norm(tf.matmul(tf.matmul(H,L),H),ord='fro',axis=[0,1])
        return trace / (HKH*HLH)
    else:
        return trace / tf.cast((n*n),tf.float32)


def gather_cols(params, indices, name=None):
    """Gather columns of a 2D tensor.

    Args:
        params: A 2D tensor.
        indices: A 1D tensor. Must be one of the following types: ``int32``, ``int64``.
        name: A name for the operation (optional).

    Returns:
        A 2D Tensor. Has the same type as ``params``.
    """
    with tf.op_scope([params, indices], name, "gather_cols") as scope:
        # Check input
        params = tf.convert_to_tensor(params, name="params")
        indices = tf.convert_to_tensor(indices, name="indices")
        try:
            params.get_shape().assert_has_rank(2)
        except ValueError:
            raise ValueError('\'params\' must be 2D.')
        try:
            indices.get_shape().assert_has_rank(1)
        except ValueError:
            raise ValueError('\'params\' must be 1D.')

        # Define op
        p_shape = tf.shape(params)
        p_flat = tf.reshape(params, [-1])
        i_flat = tf.reshape(tf.reshape(tf.range(0, p_shape[0]) * p_shape[1],
                                       [-1, 1]) + indices, [-1])
        return tf.reshape(tf.gather(p_flat, i_flat),
                          [p_shape[0], -1])

In [ ]:
# Build the computation graph for training
train_x = tf.placeholder(tf.float32, shape=[None, 48, 48, 1])
train_y = tf.placeholder(tf.float32, shape=[None])


# mean = logvar = encoder(train_x, z_dim)
# train_z, logvar = tf.split(encoder(train_x, z_dim), num_or_size_splits=2, axis=1)
# train_xr = decoder(train_z)

# train_z = reparameterize(mean, logvar)

train_z = push_forward(lidc_encoder, train_x)
train_xr = lidc_decoder(train_z)

# Build the computation graph for generating samples# Build 
gen_z = tf.placeholder(tf.float32, shape=[None, z_dim])
gen_x = lidc_decoder(gen_z, reuse=True)
print(gen_x)

pretrained_mean, pretrained_var = tf.split(lidc_encoder(train_x, z_dim, reuse=True), num_or_size_splits=2, axis=1)
# pretrained_z = reparameterize(pretrained_mean, pretrained_var)

In [ ]:
# Compare the generated z with true samples from a standard Gaussian, and compute their MMD distance
true_samples = tf.random_normal([batch_size, z_dim],stddev=Sigma)
loss_mmd = MMD(true_samples, train_z, kernel=K1, bandwidth=bandwidth1)

# loss_kl = compute_kl_loss(mean, logvar)
# loss_ksd = KSD(train_z, bandwidth=bandwidth)

# flat_train_xr = tf.reshape(train_xr, [-1, 48*48])
# flat_train_x = tf.reshape(train_x, [-1, 48*48])


# loss_nll = 0.05 * tf.reduce_mean(tf.reduce_sum(tf.square(train_xr - train_x), 1))
loss_nll = tf.reduce_mean(tf.square(train_xr - train_x))


# 50% of the time, set loss_hsic2 = 0 
hsic_signal = tf.placeholder(tf.bool)  #placeholder for a single boolean value
hsic_trigger = tf.cond(tf.equal(hsic_signal, tf.constant(True)), lambda: tf.constant(1, tf.float32), lambda: tf.constant(0, tf.float32))


# apply lambda2 on the first feature axis and discriminate all other axis with lambda3
first_axis_hsic =  HSIC(gather_cols(train_z, [0]), train_y, normalized=True)
other_axis_hsic =  HSIC(gather_cols(train_z, list(range(1,z_dim))), train_y, normalized=True)

loss_hsic = Lambda2 * first_axis_hsic - Lambda3 * other_axis_hsic 
# loss_hsic = Lambda2 * HSIC(train_z, train_y, normalized=False)

loss_hsic = hsic_trigger * loss_hsic

# loss = tf.reduce_mean(loss_nll + Lambda * loss_kl)
loss = loss_nll + Lambda1 * loss_mmd - loss_hsic
# loss = loss_nll + loss_mmd
# loss = tf.reduce_sum(loss_nll + Lambda * loss_ksd)


trainer = tf.train.AdamOptimizer(1e-3).minimize(loss)


In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('mnist_data')

# Convert a numpy array of shape [batch_size, height, width, 1] into a displayable array 
# of shape [height*sqrt(batch_size, width*sqrt(batch_size))] by tiling the images
def convert_to_display(samples):
    cnt, height, width = int(math.floor(math.sqrt(samples.shape[0]))), samples.shape[1], samples.shape[2]
    samples = np.transpose(samples, axes=[1, 0, 2, 3])
    samples = np.reshape(samples, [height, cnt, cnt, width])
    samples = np.transpose(samples, axes=[1, 0, 2, 3])
    samples = np.reshape(samples, [height*cnt, width*cnt])
    return samples

In [ ]:
sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())

saver = tf.train.Saver()

# helper function for getting batches
def next_batch(images, labels, index_in_epoch, batch_size):
    start = index_in_epoch
    # increase the index in epoch by the batch size
    index_in_epoch += batch_size
    end = index_in_epoch
    if index_in_epoch > NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN:
      end = -1
      index_in_epoch = 0
      
    return images[start:end], labels[start:end], index_in_epoch
  
def shuffle_data(images, labels):
    perm = np.arange(NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN)
    np.random.shuffle(perm)
    images = images[perm]
    labels = labels[perm]
    return images, labels
  


In [ ]:
import time
# Start training
tf.set_random_seed(100)


mmd_list = []
o_loss_list = []
rec_loss_list = []
first_hisc_list = []
other_hisc_list = []
steps_list = []


# using median heuristic bandwidth and HSIC
start_time = time.time()
index_in_epoch = 0
for i in range(steps):
    
    batch_x, batch_y, index_in_epoch = next_batch(lidc_train_x, lidc_train_y, index_in_epoch, batch_size)
    if index_in_epoch == 0:
      lidc_train_x, lidc_train_y = shuffle_data(lidc_train_x, lidc_train_y)
      batch_x, batch_y, index_in_epoch = next_batch(lidc_train_x, lidc_train_y, index_in_epoch, batch_size)
      
    batch_x = batch_x.reshape(-1, 48, 48, 1)
    use_hsic = False
    if np.random.random() <= prob_to_hsic:
        use_hsic = True
    
    _, o_loss, nll, mmd, f_hsic, o_hsic = sess.run([trainer, loss, loss_nll, loss_mmd, first_axis_hsic, other_axis_hsic], feed_dict={train_x: batch_x, train_y: batch_y,
                                                                                                     hsic_signal: use_hsic})
   
    if i % 100 == 0:
        print("Using hsic:", use_hsic)
        print("epoch: {}, Overall loss is {}, recon loss is {}, mmd loss is {}, first hsic is {}, other hsic is {}".format(
            i, o_loss, nll, mmd, f_hsic, o_hsic))
#         print("epoch: {}, Overall loss is {}, Negative log likelihood is {}, mmd loss is {}".format(
#             i, o_loss, nll, mmd))
        elapsed_time = time.time() - start_time
        print("time elapsed: {0:.2f}s".format(elapsed_time))
        start_time = time.time()
        # storing data for plot 
        mmd_list += [mmd]
        o_loss_list += [o_loss]
        rec_loss_list += [nll]
        steps_list += [i]
        first_hisc_list += [f_hsic]
        other_hisc_list += [o_hsic]
        
    if i % 1000 == 0:
        # feed in test image to get generated mmd loss
#         test_x, test_y = mnist.test.next_batch(batch_size)
        test_x, test_y = lidc_test_x[:batch_size], lidc_test_y[:batch_size]
        test_x = test_x.reshape(-1, 48, 48, 1)
#         samples, gen_mmd, my_z, fa, oa= sess.run([gen_x, loss_mmd, pretrained_mean, first_axis_hsic, other_axis_hsic],  feed_dict={gen_z: np.random.normal(size=(64, z_dim)), train_x: test_x, train_y: test_y})
        samples, gen_mmd, my_z= sess.run([gen_x, loss_mmd, pretrained_mean],  feed_dict={gen_z: np.random.normal(size=(49, z_dim)), train_x: test_x})
        plt.imshow(convert_to_display(samples * 255.), cmap='Greys_r')
        plt.grid()
        plt.show()
#         print(fa, oa)
        print("generated mmd loss: {}, my_z: {}".format(gen_mmd, my_z[0]))

# saver.save(sess, "./hsic_test.ckpt")

In [ ]:
import matplotlib.patches as mpatches
plt.style.use('default')

# plotting learning curve
# plt.ylim(0.0, 100.0)

aggregated_list = np.array(rec_loss_list) + Lambda1 * np.array(mmd_list)

# plt.plot(steps_list, aggregated_list, color='red')

# plt.xlabel("steps", fontsize=20)
# plt.ylabel("recon loss", fontsize=20)
# # plt.title("reconstruction loss vs steps")
# plt.tight_layout()

# plt.savefig("recon_loss_new.png")
# plt.show()

# plt.plot(steps_list, mmd_list, color='green')

# plt.xlabel("steps", fontsize=20)
# plt.ylabel("mmd loss", fontsize=20)

# plt.tight_layout()

# # plt.title("mmd loss vs steps")
# plt.savefig("mmd_loss_new.png")

# plt.show()

# t = np.arange(0.01, 10.0, 0.01)
# data1 = np.exp(t)
# data2 = np.sin(2 * np.pi * t)


# color = 'tab:red'
# ax1.set_xlabel('time (s)')
# ax1.set_ylabel('exp', color=color)
# ax1.plot(t, data1, color=color)
# ax1.tick_params(axis='y', labelcolor=color)


# color = 'tab:blue'
# ax2.set_ylabel('sin', color=color)  # we already handled the x-label with ax1
# ax2.plot(t, data2, color=color)
# ax2.tick_params(axis='y', labelcolor=color)

# fig.tight_layout()  # otherwise the right y-label is slightly clipped
# plt.show()


fig, ax1 = plt.subplots()

ax1.plot(steps_list, first_hisc_list, linewidth=2, color='blue')
ax1.plot(steps_list, other_hisc_list, linewidth=2, color='aquamarine')

ax1.set_xlabel("steps", fontsize=20)
ax1.set_ylabel("HSIC", fontsize=20)
ax1.set_ylim(top=1)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
ax2.plot(steps_list, aggregated_list, linewidth=2, color='magenta')
ax2.set_ylabel('loss', fontsize=20) 
# ax2.set_ylim()


blue_patch = mpatches.Patch(color='blue', label='HSIC on dep. axis')
aqua_patch = mpatches.Patch(color='aquamarine', label='HSIC on indep. axis')
magenta = mpatches.Patch(color='magenta', label='recon_loss+MMD')

plt.xlim(0, 18500)
ax1.legend(handles=[blue_patch, aqua_patch, magenta], loc=7, fontsize=14)


# plt.title("hisc on dep. axis vs steps")
plt.tight_layout()

plt.savefig("lidc_loss.png")
plt.show()

# plt.plot(steps_list, other_hisc_list, color='aquamarine')
# # adding legends 
# red_patch = mpatches.Patch(color='red', label='recon loss')
# green_patch = mpatches.Patch(color='green', label='mmd loss')
# blue_patch = mpatches.Patch(color='blue', label='hsic on dep. axis')
# aqua_patch = mpatches.Patch(color='aquamarine', label='hsic on indep. axis')

# plt.legend(handles=[red_patch, blue_patch, aqua_patch, green_patch], loc=1)

# plt.xlabel("steps")
# plt.ylabel("loss")
# plt.title("reconst, hsic and mmd loss vs steps")

# file name format
# Lambda1_Lambda2_Lambda3_ratio_accuracy.png
# acc = round(acc, 3)
# nonorm = no normalization for hsic
# plt.savefig("./hsic_data/axis1_{0}_{1}_{2}_{3}_{4:.3f}.png".format(Lambda1, Lambda2, Lambda3, 1-prob_to_hsic, acc))

In [ ]:

test_z = sess.run(pretrained_mean,  feed_dict={train_x: lidc_test_x.reshape(-1, 48, 48, 1)})

In [ ]:
depedent_axis = test_z[:, 0]
second_z = test_z[:, 1]

from sklearn import decomposition
import matplotlib.cm as cm

pca = decomposition.PCA(n_components=1)
print(test_z[:, 1:].shape)
pca.fit(test_z[:, 1:])
independent_axis_first_component = pca.transform(test_z[:, 1:])

label = lidc_test_y

# plt.style.use('default')
# plt.grid(False)

# label_patches = []

# x = np.arange(10)
# ys = [i+x+(i*x)**2 for i in range(10)]

# colors = cm.viridis(np.linspace(0, 1, len(ys)))

# c_i = 1
# for i in range(1, 6):
# #   color = sns.color_palette('viridis')[i]
#   color = colors[c_i]
#   c_i += 2
#   plt.scatter(depedent_axis[label==i], independent_axis_first_component[label==i], 
#               c=color , s=5)
#   label_patch = mpatches.Patch(label="score: {}".format(i), color=color)
#   label_patches.append(label_patch)
  
# # plt.scatter(depedent_axis, independent_axis_first_component, c=label, s=5, cmap='viridis')

# # plt.colorbar()
# plt.xlim(-2.5, 2.8)
# plt.ylim(-4.1, 4.1)

# plt.legend(handles=label_patches, loc='best')

# # plt.title("HSIC dep. axis vs HSIC indep. axis PCA")
# plt.xlabel("HSIC dep. axis", fontsize=20)
# plt.ylabel("HSIC indep. axis PCA", fontsize=20)
# plt.tight_layout()

# plt.savefig("hsicplot_new.png")




In [ ]:

all_dep_z = []
all_indep_z = []

label = np.array(label)

de_axis = depedent_axis
ind_axis = np.array([i[0] for i in independent_axis_first_component])

for i in range(1,6):
  all_dep_z.append(de_axis[label==i])
  all_indep_z.append(ind_axis[label==i])
  

all_dep_z = np.array(all_dep_z)
all_indep_z = np.array(all_indep_z)

In [ ]:
import seaborn as sns
import matplotlib.patches as mpatches
from matplotlib import gridspec


def plot_hsic_density(ax, all_z, xlim, file_name, vertial=False):
  x = np.arange(10)
  ys = [i+x+(i*x)**2 for i in range(10)]

  colors = cm.viridis(np.linspace(0, 1, len(ys)))
  label_patches = []

  c_i = 1
  score = 1

  for i in range(len(all_z)):
    color = colors[c_i]
    c_i += 2
    z = all_z[i]
    sns.kdeplot(z, color=color, vertical=vertial)
    label = format(score)
    score += 1

#   ax.xticks([])
#   ax.yticks([])
  ax.set_xticks([])
  ax.set_yticks([])
  if vertial:
    ax.set_ylim(xlim[0], xlim[1])
    ax.set_xlim(left=0)
  else:
    ax.set_xlim(xlim[0], xlim[1])
    ax.set_ylim(bottom=0)
    
def plot_hsic(ax):
  label_patches = []

  x = np.arange(10)
  ys = [i+x+(i*x)**2 for i in range(10)]

  colors = cm.viridis(np.linspace(0, 1, len(ys)))

  c_i = 1
  for i in range(1, 6):
  #   color = sns.color_palette('viridis')[i]
    color = colors[c_i]
    c_i += 2
    ax.scatter(depedent_axis[label==i], independent_axis_first_component[label==i], 
                c=color , s=5)
    label_patch = mpatches.Patch(label="score: {}".format(i), color=color)
    label_patches.append(label_patch)

  # plt.scatter(depedent_axis, independent_axis_first_component, c=label, s=5, cmap='viridis')

  # plt.colorbar()
  ax.set_xlim(-2.5, 2.8)
  ax.set_ylim(-4.2, 4.2)

  plt.legend(handles=label_patches, loc='best')

  # plt.title("HSIC dep. axis vs HSIC indep. axis PCA")
  ax.set_xlabel("HSIC dep. axis", fontsize=13.5)
  ax.set_ylabel("1st prin. comp. of HSIC indep. axis", fontsize=13.5)

fig = plt.figure(figsize=(8, 6)) 
gs = gridspec.GridSpec(2, 2, width_ratios=[3, 1], height_ratios=[1,3]) 
ax1 = plt.subplot(gs[0])
plot_hsic_density(ax1, all_dep_z, [-2.5, 2.8],"hsicplot_dep_new.png")
ax2 = plt.subplot(gs[3])
plot_hsic_density(ax2, all_indep_z, [-4.1, 4.1],"hsicplot_indep_new.png", vertial=True)
ax3 = plt.subplot(gs[2])
plot_hsic(ax3)




# fig = plt.figure(figsize = (15,8))
# ax1 = fig.add_subplot(1,2,1)
# ax2 = fig.add_subplot(1,2,2, gridspec_kw = {'width_ratios':[3, 1]})

plt.tight_layout()
plt.savefig("hsicplot_new.png")
plt.show()

print(np.std(depedent_axis))
print(np.mean(depedent_axis))
print(np.std(independent_axis_first_component))


In [ ]:
combined_axis = test_z[:,0] + ind_axis

# sorted_test_z = test_z[test_z[:,0].argsort()]
sorted_pca_index = np.argsort(-ind_axis)

a = np.argsort(combined_axis)

all_sample_z = []
y_a = -3 # y axis, -3 to 3
sample_pca = -500
while y_a <= 3.5:
  
  while sample_pca < (y_a - 0.05) or sample_pca > (y_a + 0.05):
    rd = np.random.normal(size=(1, z_dim-1))
    sample_pca = pca.transform(rd)[0][0]
    
#   print(sample_pca)  
  x_a = -2 # x axis, -1.5 to 1.5
  count = 0
  while x_a < 2:
    x_a += 0.4
    new_z = np.concatenate((np.array([x_a]), rd[0]))
    all_sample_z.append(new_z)
    
  y_a += 0.7
  


all_sample_z = np.array(all_sample_z)
print(all_sample_z.shape)


In [ ]:
samples = sess.run(gen_x,  feed_dict={gen_z: all_sample_z, train_x: test_x})

def convert_to_display(samples):
    cnt, height, width = int(math.floor(math.sqrt(samples.shape[0]))), samples.shape[1], samples.shape[2]
    samples = np.transpose(samples, axes=[1, 0, 2, 3])
    samples = np.reshape(samples, [height, cnt, cnt, width])
    samples = np.transpose(samples, axes=[1, 0, 2, 3])
    samples = np.reshape(samples, [height*cnt, width*cnt])
    return samples
  

fig = plt.figure(figsize=(8, 6)) 

plt.imshow(convert_to_display(samples[:100] *255.), cmap='Greys_r')
plt.xticks(np.arange(0, 500, step=50), [str(round(i,1)) for i in np.arange(-2, 2, step=.4)])
plt.yticks(np.arange(0, 500, step=50), reversed([str(round(i,1)) for i in np.arange(-3, 3.7, step=.7)]))


plt.grid(False)
plt.title("10x10 generated samples on HSIC dep. axis", fontsize=15)
plt.ylabel("1st prin. comp. HSIC (malig. independent)", fontsize=16)
plt.xlabel("HSIC (malig. dependent)", fontsize=16)
plt.tight_layout()
plt.savefig("generated_image_lidc.png")
plt.show()

# plt.imshow(samples[0].reshape((48, 48)), cmap='Greys_r')
# plt.show()

# plt.imshow(samples[1].reshape((48, 48)), cmap='Greys_r')
# plt.show()

# plt.imshow(samples[10].reshape((48, 48)), cmap='Greys_r')
# plt.show()

In [ ]:
from sklearn.neighbors import NearestNeighbors

flattend_train = np.array([i.flatten() for i in lidc_train_x])
flattend_samples = np.array([i.reshape((48, 48)).flatten() for i in samples])

print(flattend_train.shape)
print(flattend_samples.shape)

nbrs = NearestNeighbors(n_neighbors=3, algorithm='ball_tree').fit(flattend_train)

In [ ]:
import pandas as pd
distances, indices = nbrs.kneighbors(flattend_samples)

score_matrix = []
tmp = []
for i in range(len(flattend_samples)):
  mean_score = np.mean(lidc_train_y[indices[i]])
  if (i % 10 == 0 and i != 0):
    score_matrix.append(np.array(tmp))
    tmp = []
  tmp.append(mean_score)

score_matrix = np.array(score_matrix)
# print(score_matrix)

score_matrix.shape

score_mean_list = []
score_var_list = []
for i in range(0, 10):
  score_mean_list.append(round(np.mean([s[i] for s in score_matrix]), 2))
  score_var_list.append(round(np.std([s[i] for s in score_matrix]), 2))

print(score_mean_list) 
print(score_var_list)

score_mean_list.reverse()
x, y = pd.Series(np.arange(-2, 2, step=.4), name="HSIC dep. axis"), pd.Series(score_mean_list, name="average malig. score")
sns.regplot(x=x, y=y, color ='blue')

# plt.title("Avg malig. score of generated samples with 3 NNs", fontsize=14)
plt.xlabel("HSIC dep. axis", fontsize=18)
plt.ylabel("mean malig. score", fontsize=18)

plt.tight_layout()
plt.savefig("sample_score.png")

